In [1]:
import pandas as pd

### 2017 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2017_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2016
start_year = 2017
target_year = 2018

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [5]:
# Initialize an empty DataFrame for the master data
master_2017 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.endswith("_HRM2"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=0)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'Rechnung': f'Rechnung {previous_year}',
            'Budget': f'Budget {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget.1': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Rename the first column to 'HRM 2' and third column to "in 1000 CHF"
        df.columns.values[0] = 'HRM2-ID'
        df.columns.values[2] = 'HRM2-Bezeichnung'

        # Delete unecessary columns
        df.drop(df.columns[1], axis=1, inplace=True)
             
        # Add a new column with the DataFrame name (sheet name)
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split('_HRM2').str[0]
        df['Year'] = 2017

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2017 = pd.concat([master_2017, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2017.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2016,Budget 2017,Rechnung 2017,Budget 2018,Source,Kanton,Year,Unnamed: 7,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,NaN,NaN,NaN,ZH_HRM2,ZH,2017,NaN,NaN,NaN
1,30,Personalaufwand,5086778.72952,5233947.21600,5240375.77332,5315346.18700,ZH_HRM2,ZH,2017,NaN,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2958331.72029,3085081.57300,3035765.80313,3119151.58300,ZH_HRM2,ZH,2017,NaN,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,200856.35723,198576.90000,210278.89938,195508.10000,ZH_HRM2,ZH,2017,NaN,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,-705.51857,3259.20000,8031.84811,3405.40000,ZH_HRM2,ZH,2017,NaN,NaN,NaN


### Translating FR columns to DE

In [6]:
# # Display rows where the column 'Kanton' is 'HRM2_KT_VD'
# filtered_rows = master_2017[master_2017['Kanton'] == 'VD_HRM2']

# # Display the filtered rows
# filtered_rows.head()

In [7]:
# 1. Update 'Rechnung 2016' based on 'Compte'
master_2017['Rechnung 2016'] = master_2017.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2016']) and not pd.isna(row['Compte']) else row['Rechnung 2016'],
    axis=1
)

# 2. Update 'Rechnung 2017' based on 'Compte.1'
master_2017['Rechnung 2017'] = master_2017.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2017']) and not pd.isna(row['Compte.1']) else row['Rechnung 2017'],
    axis=1
)

# Drop the column named 'Unnamed: 7'
master_2017.drop('Unnamed: 7', axis=1, inplace=True)

# Display the first few rows of the updated DataFrame
master_2017.head()


,HRM2-ID,HRM2-Bezeichnung,Rechnung 2016,Budget 2017,Rechnung 2017,Budget 2018,Source,Kanton,Year,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,NaN,NaN,NaN,ZH_HRM2,ZH,2017,NaN,NaN
1,30,Personalaufwand,5086778.72952,5233947.21600,5240375.77332,5315346.18700,ZH_HRM2,ZH,2017,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2958331.72029,3085081.57300,3035765.80313,3119151.58300,ZH_HRM2,ZH,2017,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,200856.35723,198576.90000,210278.89938,195508.10000,ZH_HRM2,ZH,2017,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,-705.51857,3259.20000,8031.84811,3405.40000,ZH_HRM2,ZH,2017,NaN,NaN


In [8]:
# # Display rows where the column 'Kanton' is 'VD_HRM2'
# filtered_rows = master_2017[master_2017['Kanton'] == 'VD_HRM2']

# # Display the filtered rows
# filtered_rows.head()

### Before saving the data, we drop non-numeric "HRM2-IDs", because they would cause issues with subsequent automated analysis...

In [9]:
# Drop rows where 'HRM-ID' values are non-numeric
master_2017 = master_2017[pd.to_numeric(master_2017['HRM2-ID'], errors='coerce').notna()]

# Display the first few rows of the updated DataFrame
len(master_2017)

2704

In [10]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2017_data.csv'

# Save the DataFrame to a CSV file
master_2017.to_csv(output_path, index=False)
